In [ ]:
import copy
import random
import distutils.core
import cv2
import torch
import sys
import os
from PIL import Image
import numpy as np
import shutil
import glob


import argparse


# include the utils folder in the path

from datetime import datetime as dt


from os import listdir
from os.path import isfile, join


In [ ]:

detections_root_path = "./outputs/detections/"
sequence= "S01"
track_eval_root_path = "../TrackEval/"
#gt path 
gt_path = "../datasets/AITrack/train/"

filenames = [f for f in listdir(detections_root_path) if isfile(join(detections_root_path, f))]
current_sequence_filenames = [s for s in filenames if sequence in s]
current_sequence_seqini = [s for s in current_sequence_filenames if "seqini" in s]
current_sequences= [s for s in current_sequence_filenames if "seqini" not in s]
print(current_sequence_filenames)
print(current_sequence_seqini)
print(current_sequences)

#tracker save path
track_eval_trackers_path = track_eval_root_path+"/data/trackers/mot_challenge/"+sequence+"-train/all_cams/data/"
seq_tracker_path = track_eval_trackers_path+"/triplet_nn/data/"

#trackeval gt save path
track_eval_gt_path = track_eval_root_path+"/data/gt/mot_challenge/"+sequence+"-train/all_cams/gt/"
track_eval_seqinfo_path = track_eval_root_path+"/data/gt/mot_challenge/"+sequence+"-train/all_cams/seqinfo.ini"
track_eval_gt_path_seqmaps=track_eval_root_path+"/data/gt/mot_challenge/seqmaps/"

#seqmap path 
track_eval_seqmap_path = track_eval_root_path+"/data/gt/mot_challenge/seqmaps/"+sequence+"-train.txt"
seqmap_content = ""

#create dets folder
if not os.path.exists(track_eval_trackers_path):
    print("Creating directory ", track_eval_trackers_path)
    os.makedirs(track_eval_trackers_path)

#create gt and seqmap file
if not os.path.exists(track_eval_gt_path):
    print("Creating directory ", track_eval_gt_path)
    os.makedirs(track_eval_gt_path)

with open(track_eval_gt_path_seqmaps+"/"+sequence+"-train.txt", "w") as f:
    f.writelines("name\nall_cams\n")

merged_gt = []
merged_dets = []
frame_offset  = 0
for cam_file in current_sequences:
    cam = cam_file.split("_")[1]
    cam = cam.split(".")[0]
    print("Writing ", cam, " detections")
    #open gt path
    old_gt_filepath = gt_path+"/"+sequence+"/"+cam+"/gt/gt.txt"

    # Open the file and read its contents
    with open(old_gt_filepath, "r") as f:
        lines_gt = f.readlines()
    with open(detections_root_path+cam_file, "r") as f:
        lines_dets = f.readlines()
    #print("LINES ", lines)
    # overwrite gt file frames
    for line in lines_gt:
        frame_num = str(int(line.split(",")[0]) + frame_offset)
        print("Curr frame", frame_num)
        merged_line = frame_num + "," + line.split(",", 1)[1]
        merged_gt.append(merged_line)
    #overwrite detection frames
    for line in lines_dets:
        frame_num = str(int(line.split(",")[0]) + frame_offset)
        print("Curr frame", frame_num)
        merged_line = frame_num + "," + line.split(",", 1)[1]
        merged_dets.append(merged_line)
    current_cam_seqini = [s for s in current_sequence_seqini if cam in s][0]
    current_seqini_old_path = detections_root_path+current_cam_seqini
    with open(current_seqini_old_path, "r") as f:
        lines_seqini = f.readlines()
    for line in lines_seqini:
        if "seqLength" in line:
            frame_length = line.split("=")[1]
    
            
    frame_offset = int(frame_length)+ frame_offset
    
    print(frame_offset)

# Write the merged lines to a new txt file
with open(track_eval_gt_path+"gt.txt", "w") as f:
    f.writelines(merged_gt)
    
with open(track_eval_trackers_path+"all_cams.txt", "w") as f:
    f.writelines(merged_dets)

seqinfo = "[Sequence]\nname=all_cams\nimDir=img1\nframeRate=30\nseqLength="+str(frame_offset)+"\nimWidth=1920\nimHeight=1080\nimExt=.jpg"

with open(track_eval_seqinfo_path, "w") as f:
    f.writelines(seqinfo)

In [5]:
output = os.popen("python ../TrackEval/scripts/run_mot_challenge.py  --BENCHMARK S01 --DO_PREPROC False ").read()
print(output)

  Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
You should consider upgrading via the '/home/neru/.local/bin/.virtualenvs/cv/bin/python -m pip install --upgrade pip' command.

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /home/neru/M6/w3/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

MotChallenge2DBox Config:
PRINT_CONF